In [43]:
# Install dependencies. You only need to run this cell once at startup.
# !pip install https://github.com/tingelst/tpk4170-robotics/archive/master.zip pycollada
#!pip install /Users/lars/devel/tpk4170-robotics/

# Exercise 6 - TPK4170 Robotics


**Topic:** Inverse kinematics

**Deadline:** October 19, 2018


In [44]:
import numpy as np
np.set_printoptions(suppress=True, precision=6)

In [45]:
from exercise_6 import fk, Jacobian, UR5Visualizer

# Inverse kinematics for the UR5

<img src="https://aws.roboticsbusinessreview.com/wp-content/uploads/2014/09/UR5_Robot02.jpg" width="300"/>

## Forward kinematics

The Denavit-Hartenberg parameters for the UR5 are given below:

In [3]:
a = [0.00000, -0.42500, -0.39225,  0.00000,  0.00000,  0.0000]
d = [0.089159,  0.00000,  0.00000,  0.10915,  0.09465,  0.0823]
alpha = [1.570796327, 0, 0, 1.570796327, -1.570796327, 0. ]
q_zero_offset = [0., 0., 0., 0., 0., 0.]

The `fk` function implements the forward kinematics of the UR5 given the 6-vector of joint angles `q`. The function returns a tuple of six homogenous transformation matrices: $\mathbf{T}^0_1,\ \mathbf{T}^0_2,\ \mathbf{T}^0_3,\ \mathbf{T}^0_4,\ \mathbf{T}^0_5,\ \mathbf{T}^0_6$.

In [4]:
q_home = np.array([0,-np.pi/2, np.pi/2,0,0,0])
Ts = fk(q_home)
T01, T02, T03, T04, T05, T06 = Ts
print(T06)

[[ 1.        0.        0.       -0.39225 ]
 [ 0.       -0.       -1.       -0.19145 ]
 [ 0.        1.       -0.        0.419509]
 [ 0.        0.        0.        1.      ]]


## Jacobian

The `Jacobian` function takes as input a tuple of six homogeneous transformation matrices and returns the Jacobian matrix for a manipulator with six revolute joints.

In [5]:
J = Jacobian(Ts)
print(J)

[[ 0.19145 -0.33035  0.09465  0.09465 -0.0823  -0.     ]
 [-0.39225  0.       0.       0.       0.       0.     ]
 [ 0.      -0.39225 -0.39225  0.      -0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.      -1.      -1.      -1.       0.      -1.     ]
 [ 1.      -0.      -0.      -0.      -1.      -0.     ]]


## Visualizer

The `UR5Visualizer` class implement a visualization of the UR5 with frames attached to each link. The `show` method takes as input a 6-vector `q` of joint angles and visualize the current pose. The `interact` method can be used to *jog* each joint.

In [6]:
vis = UR5Visualizer()

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [7]:
vis.show(q_home)

In [8]:
vis.interact()

interactive(children=(FloatSlider(value=0.0, description='q1', max=3.141592653589793, min=-3.141592653589793),…

## Task 1

Following section *10.8 Inverse kinematics of the UR5 industrial robot* in *A note on robot kinematics*, implement the analytic inverse kinematics of the UR5 in the function `analytic_ik`. The function should take at least the desired pose as a homogenous transformation matrix as input and return a 6-vector of joint angles.

In [46]:
def ik_analytical(T, shoulder=1, elbow=1, wrist=1):
    
    # Implement this function
    
    return q  

## Task 2
Implement the iterative inverse kinematics function `iterative_ik` for the UR5. The function should take at least the desired pose as a homogenous transformation matrix and the initial joint angles as input and return a 6-vector of joint angles. Use either the analytical Jacobian or the geometric Jacobian.

In [10]:
def ik_iterative(T, q0):
    
    # Implement this function
    
    return q

## Evaluate your solution

In [31]:
# Desired pose
Td = np.array([[ 1.      ,  0.      ,  0.      , -0.244156],
               [ 0.      , -0.      , -1.      , -0.664567],
               [-0.      ,  1.      , -0.      ,  0.352134],
               [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [32]:
# Initial guess
q0 = q_zero = np.array([1,1,1,1,1,1])

In [33]:
# Solution from the analytical IK
q_analytical = ik_analytical(Td)
print(q_analytical)

[ 1.       -0.999999  1.        6.283184  1.       -0.      ]


In [34]:
# Solution from the iterative IK
q_iterative = ik_iterative(Td, q0)
print(q_iterative)

[ 1.       -0.999999  1.        6.283184  1.       -0.      ]


In [38]:
# Pose, given the analytical solution
T_analytical = fk(q_analytical)[5]
print(T_analytical)

[[ 1.        0.        0.       -0.244156]
 [ 0.        0.000001 -1.       -0.664567]
 [-0.        1.        0.000001  0.352134]
 [ 0.        0.        0.        1.      ]]


In [39]:
# Pose, given the iterative solution
T_iterative = fk(q_iterative)[5]
print(T_iterative)

[[ 1.        0.        0.       -0.244156]
 [ 0.        0.000001 -1.       -0.664567]
 [-0.        1.        0.000001  0.352134]
 [ 0.        0.        0.        1.      ]]


In [40]:
# Check if the computed poses are equal. Note that this is just a simple check. 
# You should in general compare the position and orientation separately, and decompose
# the rotation angle and rotation axis. The result should be True.
np.allclose(T_analytical, T_iterative)

True

In [41]:
def ik_iterative(trf, q0=np.array([1,1,1,1,1,1])):
    Rd = trf[:3,:3]
    qk = q0
    qs = []
    for i in range(1000):
        Ts = fk(qk)
        Tk = Ts[5]
        Jk = Jacobian(Ts)
        Rk = Tk[:3,:3]
        Re = Rd @ Rk.T
        ep = trf[:3,3] - Tk[:3,3]
        eo = np.array([Re[2,1] - Re[1,2], 
                       Re[0,2]-  Re[2,0], 
                       Re[1,0] - Re[0,1]]) * 0.5

        e = np.array((ep, eo)).reshape(6,1)

        K = 0.1
        dq = K * np.dot(np.linalg.pinv(Jk),e).ravel()
        qk = qk + dq 
        qs.append(qk)

        if np.linalg.norm(e) < 1e-6:
            break
    return qk

In [48]:
ik_analytical = ik_iterative

In [47]:
def ik_analytical(trf):
    pass 